In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from dotenv import dotenv_values

In [2]:
config = dotenv_values('../.env')

In [3]:
train = pd.read_parquet(config["ENGINEERED_DATA"] + "train_fe.parquet")
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,...,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0.933824,0.024194,0.868580,0.960384,0.934745,0,0.230769,0.832050,...,1,13,-1,-1,1,13,6,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.929122,0.899820,0.022119,0.861109,0.929122,0.880519,13,7.153846,6.743468,...,1,13,-1,-1,1,13,6,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.876615,0.878454,0.028911,0.797670,0.904482,0.880875,0,0.000000,0.000000,...,1,13,-1,-1,1,13,6,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.567442,0.598969,0.020107,0.567442,0.623392,0.621776,9,1.538462,3.017046,...,1,13,-1,-1,1,13,2,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.936842,0.891679,0.042325,0.805045,0.940382,0.871900,0,0.000000,0.000000,...,1,13,1,1,1,13,6,6,1,0


In [4]:
train.shape

(458913, 1462)

In [5]:
train = train.drop("customer_ID", axis=1)
train.head()

,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,...,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target
0,0.938469,0.933824,0.024194,0.868580,0.960384,0.934745,0,0.230769,0.832050,0,...,1,13,-1,-1,1,13,6,6,1,0
1,0.929122,0.899820,0.022119,0.861109,0.929122,0.880519,13,7.153846,6.743468,0,...,1,13,-1,-1,1,13,6,6,1,0
2,0.876615,0.878454,0.028911,0.797670,0.904482,0.880875,0,0.000000,0.000000,0,...,1,13,-1,-1,1,13,6,6,1,0
3,0.567442,0.598969,0.020107,0.567442,0.623392,0.621776,9,1.538462,3.017046,0,...,1,13,-1,-1,1,13,2,3,3,0
4,0.936842,0.891679,0.042325,0.805045,0.940382,0.871900,0,0.000000,0.000000,0,...,1,13,1,1,1,13,6,6,1,0


In [9]:
train = train.groupby('target').apply(pd.DataFrame.sample, frac=0.2).reset_index(drop=True)

In [10]:
train.shape

(91783, 1461)

In [11]:
label = 'target'
eval_metric='log_loss'
save_path=config["AG_MODELS"]+"model_4"

In [12]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, 
                             path=save_path, verbosity=3).fit(
                                train, presets='best_quality', time_limit=3600,
                                ag_args_fit={'num_gpus':1}, auto_stack=True
                            )

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}, 'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Saving E:/datasets/amex-default-prediction/models/ag_models/model_4\learner.pkl
Saving E:/

		('float64', 'float') : 344 | ['P_2_std', 'D_39_mean', 'D_39_std', 'B_1_std', 'B_2_std', ...]
		('int16', 'int')     :  45 | ['D_39_first', 'D_39_min', 'D_39_max', 'D_39_last', 'B_4_first', ...]
		('int64', 'int')     :  22 | ['B_30_count', 'B_30_nunique', 'B_38_count', 'B_38_nunique', 'D_114_count', ...]
		('int8', 'int')      : 397 | ['D_44_first', 'D_44_min', 'D_44_max', 'D_44_last', 'R_2_first', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 995 | ['P_2_first', 'P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', ...]
		('int', [])       : 373 | ['D_39_first', 'D_39_min', 'D_39_max', 'D_39_last', 'D_44_first', ...]
		('int', ['bool']) :  91 | ['R_2_first', 'R_2_min', 'R_2_max', 'R_2_last', 'S_6_first', ...]
	9.5s = Fit runtime
	1459 features in original data used to generate 1459 features in processed data.
	Train Data (Processed) Memory Usage: 552.44 MB (11.2% of available memory)
Data preprocessing and feature engineering runtime = 10.25s ..

[50]	valid_set's binary_logloss: 0.258942
[100]	valid_set's binary_logloss: 0.238168
[150]	valid_set's binary_logloss: 0.234016
[200]	valid_set's binary_logloss: 0.232483
[250]	valid_set's binary_logloss: 0.231516
[300]	valid_set's binary_logloss: 0.231058
[350]	valid_set's binary_logloss: 0.231174
[400]	valid_set's binary_logloss: 0.230998
[450]	valid_set's binary_logloss: 0.231117
[500]	valid_set's binary_logloss: 0.231191


	Fitting S1F2 with 'num_gpus': 1, 'num_cpus': 6
	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True, 'device': 'gpu'}


[50]	valid_set's binary_logloss: 0.25661
[100]	valid_set's binary_logloss: 0.234511
[150]	valid_set's binary_logloss: 0.230028
[200]	valid_set's binary_logloss: 0.228042
[250]	valid_set's binary_logloss: 0.227648
[300]	valid_set's binary_logloss: 0.227507
[350]	valid_set's binary_logloss: 0.227548
[400]	valid_set's binary_logloss: 0.227521


...

[50]	valid_set's binary_logloss: 0.27611
[100]	valid_set's binary_logloss: 0.233757
[150]	valid_set's binary_logloss: 0.22396
[200]	valid_set's binary_logloss: 0.220423
[250]	valid_set's binary_logloss: 0.219289
[300]	valid_set's binary_logloss: 0.218761
[350]	valid_set's binary_logloss: 0.218651
[400]	valid_set's binary_logloss: 0.219267
[450]	valid_set's binary_logloss: 0.219596


	Fitting S1F8 with 'num_gpus': 1, 'num_cpus': 6
	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 5, 'device': 'gpu'}


[50]	valid_set's binary_logloss: 0.280672
[100]	valid_set's binary_logloss: 0.239687
[150]	valid_set's binary_logloss: 0.230613
[200]	valid_set's binary_logloss: 0.227773
[250]	valid_set's binary_logloss: 0.226909
[300]	valid_set's binary_logloss: 0.226799
[350]	valid_set's binary_logloss: 0.227354
[400]	valid_set's binary_logloss: 0.228079


Saving E:/datasets/amex-default-prediction/models/ag_models/model_4\models\LightGBMLarge_BAG_L1\utils\oof.pkl
Saving E:/datasets/amex-default-prediction/models/ag_models/model_4\models\LightGBMLarge_BAG_L1\model.pkl
	-0.2257	 = Validation score   (-log_loss)
	939.46s	 = Training   runtime
	2.19s	 = Validation runtime
Saving E:/datasets/amex-default-prediction/models/ag_models/model_4\models\trainer.pkl
Not enough time left to finish repeated k-fold bagging, stopping early ...
Completed 1/20 k-fold bagging repeats ...
Loading: E:/datasets/amex-default-prediction/models/ag_models/model_4\models\LightGBMXT_BAG_L1\utils\oof.pkl
Loading: E:/datasets/amex-default-prediction/models/ag_models/model_4\models\LightGBM_BAG_L1\utils\oof.pkl
Loading: E:/datasets/amex-default-prediction/models/ag_models/model_4\models\CatBoost_BAG_L1\utils\oof.pkl
Loading: E:/datasets/amex-default-prediction/models/ag_models/model_4\models\LightGBMLarge_BAG_L1\utils\oof.pkl
Model configs that will be trained (in ord

RuntimeError: expected scalar type Long but found Int

In [13]:
results=predictor.fit_summary()

NameError: name 'predictor' is not defined